In [1]:
import sys
sys.path.append(r"C:\Users\WilcoSievers\OneDrive - MOYO Business Advisory\Documents\GitHub\python-util\excel-util\\" )
import exutil

In [72]:
import pandas
import openpyxl
import numpy
import fnmatch
import datetime
import time

In [3]:
def search_for_columns(file_paths : list[str], target_sheets : list[str], start_points : list , end_points : list[str] = ['NaN'], **kwargs):
    """
    Takes in 
    
    *Args:
        
        file_paths = List of file paths that need to be searched
        
        target_sheets = Sheet name to target, if no hit is recieved for first val continue to next val and search unit hit.
        
        start_points = List[str] of multiple values to search. These points are used as an anchor points for when selecting values. If the first value in the iterable returns None, move onto the next value.
        
        end_points = List[str] of multiple values to search. These points are used as an anchor points for when selecting values. If the first value in the iterable returns None, move onto the next value.

    **kwargs:
        
        start_pivot = Tuple of (int, int) as (row, col) - used to pivot from start point 
        
        end_pivot = Tuple of (int, int) as (row, col) - used to pivot from end point 
        
        searched_columns = Known columns of previous search results  
        
        searched_files = list[str] - list of files that have been searched and values that have
        
        start_default = (row, col) - Start value to default to if no resutls are found. If  'start_default' is not given, and the start points give no result, an error will be raised.
        
        end_default = (row, col) - Start value to default to if no resutls are found. If  'start_default' is not given, and the start points give no result, an error will be raised
        
        columns_to_exclue = Columns to remove, as there are certain cell values that are just thrown around in the bloody sheet.
    """
    # import dependant functions
    from openpyxl import load_workbook
    from exutil import is_similar, search_excl_val, exc_coord_to_rc, get_excl_sheet_vals, filter_files
    from datetime import datetime

    
    # Prelude variables used for various downstream processes
    start_point, end_point = {'row' : 0, 'col' : 0}, {'row' : 0, 'col' : 0};
    start_point_pivot, end_point_pivot = {'row' : 0, 'col' : 0}, {'row' : 0, 'col' : 0};
    start_point_default, end_point_default = {'row' : 0, 'col' : 0}, {'row' : 0, 'col' : 0};
    start_default, end_default = {'row' : None, 'col' : None}, {'row' : None, 'col' : None};
    
    column_result = [];
    column_source = [];
    searched_files = [];
    columns_to_exclue = [];

    
    for key, val in kwargs.items():
        match key:
            case 'start_pivot':
                start_point_pivot['row'], start_point_pivot['col'] = val;
            case 'end_pivot':
                end_point_pivot['row'], end_point_pivot['col'] = val;
            case 'searched_files':
                searched_files = searched_files + val; # append the files
            case 'start_default':
                start_point_default['row'], start_point_default['col'] = val;
            case 'end_default':
                end_point_default['row'], end_point_default['col'] = val;
            case 'searced_columns':
                column_result = searched_columns;
            case 'columns_to_exclue':
                columns_to_exclue = val;
            case _:
                raise NameError(f"Unkown values '{key}' - '{val}' fed. Expected **kwargs = [start_pivot, end_pivot, searched_files, start_default, end_default]")
    
    # remove file paths already searched
    files_paths_to_search = file_paths;


    
    # this is inneficient method. Search throygh file paths list and filter out values that have been searched
    for file_i in searched_files:
        file_slice_idx = file_i.rfind('\\') if file_i.rfind('\\')  != -1 else 0;
        files_paths_to_search = filter_files(files_paths_to_search,                    # the path list to iterate and make shrink
                                             '*' + file_i[file_slice_idx::] + '*',     # the pattern to search for, just look at final file
                                             matchPattern = False)

    
    for file_i in files_paths_to_search:
        try: 
            active_workbook = load_workbook(file_i);
        except PermissionError as pe:
            print(pe); # a file is open in another app, thus permission is denied, continue on
            continue;

        # Search for most relevenat sheet
        active_sheet_name = None;
        for sheet_ii in target_sheets:
            similar_sheets, simalr_sheet_scores = is_similar(sheet_ii, active_workbook.sheetnames, return_similarrity_score=True);
            if similar_sheets is not None:
                active_sheet_name = similar_sheets[simalr_sheet_scores.argmax()];
                break;
        if active_sheet_name is None:
            print(f"Could not find sheet targets : {target_sheets} for {file_i}");
            continue;
        

        # set active sheet
        active_sheet = active_workbook[active_sheet_name];

        
        # search for start anchor pivot point
        for start_pii in start_points:
            excl_coord, _ = search_excl_val(active_sheet, [start_pii], return_first_hit = True);
            if excl_coord is not None:
                start_point['row'], start_point['col'] =  exc_coord_to_rc(excl_coord);
                start_point['row'] = start_point['row'] + start_point_pivot['row']; 
                start_point['col'] = start_point['col'] + start_point_pivot['col']; 
                break;
        # Ensure start point does not have any errors
        if (start_point['row'] == 0) and (start_point['col'] == 0):
            if (start_point_default['row'] is None) and (start_point_default['col'] is None):
                raise UserWarning(f"Could not find start values '{start_points}' in '{active_sheet_name}' in '{file_i}'")
            else:
                start_point = start_point_default;

                
        # search for end anchor pivot point
        for end_pii in end_points:
            excl_coord, _ = search_excl_val(active_sheet, [end_pii], return_first_hit = True);

            if excl_coord is not None:
                end_point['row'], end_point['col'] =  exc_coord_to_rc(excl_coord);
                end_point['row'] += end_point_pivot['row']; 
                end_point['col'] += end_point_pivot['col']; 
                break;
        # Ensure end point does not have any errors
        if (end_point['row'] == 0) and (end_point['col'] == 0):
            if (end_point_default['row'] is None) and (end_point_default['col'] is None):
                raise UserWarning(f"Could not find end values '{end_points}' in '{active_sheet_name}' in '{file_i}'")
            else:
                end_point =  end_point_default;

        # extract the values, columns are in signle line, thus use start col as singular column
        extracted_columns = get_excl_sheet_vals(active_sheet,   # columns are in singular column
                                               (start_point['row'], start_point['col']),
                                               (end_point['row'], start_point['col']));

        # remove any None/ empty values
        extracted_columns = [colVal for colVal in extracted_columns if colVal]; 
        # rek0ve date values
        extracted_columns = [colVal for colVal in extracted_columns if not isinstance(colVal, datetime)]; 
        # remove any weird values
        extracted_columns = [colVal for colVal in extracted_columns if (str(colVal).find('=') == -1) and str(colVal).find(',') == -1];

        
        for new_col_ii in extracted_columns:
            if new_col_ii not in column_result:
                column_result.append(new_col_ii);
                column_source.append(file_i);

    # final cleanup for values
    col_source_list = [];
    col_list = [];
    for col_val_ii, path_source_ii in zip(column_result, column_source):
        if col_val_ii not in columns_to_exclue:
            col_source_list.append(path_source_ii[path_source_ii.rfind('\\')::]);   # just take the final column/ slice
            col_list.append(col_val_ii);
            
    return col_list, col_source_list;

In [4]:
def add_new_data(data : pandas.DataFrame, files_to_search : list[str], start_col, end_col, columns_to_extract):
    return;

In [5]:
base_path = r"C:\Users\WilcoSievers\OneDrive - MOYO Business Advisory\Documents\Assigments\2025-07-04 - Mine data analysis\Data\Combine data\KPI FILE 19 Oct 24"
file_pattern = "*.xlsx"
all_file_paths = exutil.filter_files(base_path, file_pattern)


# Values for columns searcg
sheet_targets = ["densities & reagents", "densities and reagents"];
start_points = ["SP"];
end_points = ["AG HT %CaF2", "HT AVG"]; 

known_column_list = [1163, 1164, 'CRT', 'ARF', 'CL1 Feed', 'CL2 Feed', 'CL2 concentrate', 'CL3 concentrate', 'CL4 concentrate', 'CL5 concentrate', 'Refloat feed', 'Refloat 1 concentrate', 'Refloat 2 concentrate', 'Refloat 3 concentrate', 'AR Collector ', 'CL1 Collector ', 'Collector (CCT) ', 'CL1 Dextrin / CL2', 'CL2 Dextrin ', 'CL3 Dextrin ', 'CL4 Dextrin ', 'Coagulant ', 'RF2 Dextrin', 'CL5 Dispersant', 'Surge Tank Density', 'WHIMS Feed Density', 'Tails Flocculant Flow(L/min)', 'Flocculant Strength', 'Soda ash flow', 'Metspar concentrate', 'Surge Tank Density(SCADA)', 'Conditioner 1(Manual)', 'Conditioner 1', 'FLOC WTP1', 'FLOC WTP2', 'Coag WTP1', 'Coag WTP2', 'Flocculant', 'Flocculant Dosing Strength', ' ', 'CL6 concentrate(SCADA)', 'AR Dextrin']

In [79]:
data_original = pandas.read_csv("Avg_density_and_reactants_2023_2024_2025_col.csv");

previously_searched_files = pandas.read_csv("searched_files.csv").File;


In [80]:
unsearched_file_paths = [];

for file_searched in all_file_paths:
    if fnmatch.filter(searched_files, '*'+file_searched[file_searched.rfind('\\'):]):      # check if serched file is in all the list
        continue;                                              # if not, append it 
    else:
        unsearched_file_paths.append(file_searched);


In [83]:
A, b = search_for_columns(all_file_paths, sheet_targets, 
                           start_points, end_points,
                           start_pivot=(0,-3), end_pivot=(-1,0), 
                           searched_files =  previously_searched_files.to_list())

In [10]:
# new_data = add_new_data(data_original, files_to_add, path_to_save, );


In [11]:
data = data_original;
files_to_search = unsearched_file_paths;

columns_to_search = known_column_list;
start_col_searchval = datetime.time(7);
end_col_searchval  = datetime.time(5);

In [73]:
t0 = time.time();

from openpyxl import load_workbook
from exutil import is_similar, search_excl_val, exc_coord_to_rc, get_excl_sheet_vals, filter_files
from numpy import mean
# from datetime import datetime


buffer_list = [];

buffer_dict_empty = {str(col) : None for col in columns_to_search};
buffer_dict_empty['Date'] = None;
buffer_dict_empty['File'] = None;

buffer_dict = buffer_dict_empty.copy();

for file_i in files_to_search:
    try: 
        active_workbook = load_workbook(file_i);
    except PermissionError as pe:
        print(pe);                         # file cannot be accessed, thus permission is denied, continue on to next file
        continue;
        
    # Search for most relevenat sheet
    active_sheet_name = None;
    for sheet_ii in sheet_targets:
        similar_sheets, simalr_sheet_scores = is_similar(sheet_ii, active_workbook.sheetnames, return_similarrity_score=True);
        if similar_sheets is not None:
            active_sheet_name = similar_sheets[simalr_sheet_scores.argmax()];
            break;
    if active_sheet_name is None:
        print(f"Could not find sheet targets : {target_sheets} for {file_i}");
        continue;
    # set active sheet
    active_sheet = active_workbook[active_sheet_name];


    
    # search for the column coordinates values where data is stored
    try:
        # search for time as time val
        start_coordinate, _ = search_excl_val(active_sheet, [start_col_searchval]); 
    except TypeError:
        try:
            # search for time as datetime val
             start_coordinate, _  = search_excl_val(active_sheet, [datetime.datetime.combine(datetime.date(1900,1,1), start_col_searchval)]);
        except TypeError:
            # search for time as string val
            start_coordinate, _ = search_excl_val(active_sheet, [start_col_searchval.strftime("%Hh%M")]);
    try:
        # search for time as time val
        end_coordinate, _   = search_excl_val(active_sheet, [end_col_searchval]);   
    except TypeError:
        try:
            # search for time as datetime val
            end_coordinate, _  = search_excl_val(active_sheet, [datetime.datetime.combine(datetime.date(1900,1,1), end_col_searchval)]);
        except TypeError:
            # search for time as string val
            end_coordinate, _  = search_excl_val(active_sheet, [end_col_searchval.strftime("%Hh%M")]);

    
    if isinstance(start_coordinate, str):
        _, start_col = exc_coord_to_rc(start_coordinate);
    if isinstance(end_coordinate, str):
        _, end_col = exc_coord_to_rc(end_coordinate);
    # The time format changes across sheets for some reasons, thus also search for time with form "14h00" when 14:00 doesn't
    # give a hit. This is a quick workaround to get the correct column values 
    if (end_col-start_col)==1 or (): # check if column sare the same, which is a n indicaion of 
        start_coordinate, _ = search_excl_val(active_sheet, [start_col_searchval.strftime("%Hh%M")])
        end_coordinate, _   = search_excl_val(active_sheet, [end_col_searchval.strftime("%Hh%M")]);       
        if isinstance(start_coordinate, str):
            _, start_col = exc_coord_to_rc(start_coordinate);       
        if isinstance(end_coordinate, str):
            _, end_col = exc_coord_to_rc(end_coordinate);
    # more error handling will be needed, however, for now, this i sok


    # Time to extract the data
    for var_to_search in columns_to_search:
        coordinate, _ = search_excl_val(active_sheet, [var_to_search])
        numeric_vals = [];
        values_extracted = [];
        if coordinate:
            row_select, _ = exc_coord_to_rc(coordinate);
            values_extracted = get_excl_sheet_vals(active_sheet, (row_select, start_col), (row_select, end_col));
            # extract numeric vals

            for val in values_extracted:
                try:
                    numeric_vals.append(float(val));
                except (ValueError, TypeError):
                    continue;         
            mean_val =  mean(numeric_vals) if (numeric_vals != []) else 0;  # give mean result, if  numeric vals is not empty   
        else:
            mean_val = 0;
        # print(f"{var_to_search} - {mean_val} - {coordinate} - {values_extracted} - {numeric_vals}")
        buffer_dict[str(var_to_search)] = mean_val;
    
    # Extract all the date values
    date_values = search_excl_val(active_sheet, [datetime.date(1990,1,1)], return_first_hit=False, return_all_vals=True, threshold=0.1);
    
    buffer_dict['Date'] = max(date_values['Value']); # append the most up to date values, which is likely the max
    buffer_dict['File'] = file_i;
    # append the values to a list comprised of dicts
    buffer_list.append(buffer_dict.copy());



[Errno 13] Permission denied: 'C:\\Users\\WilcoSievers\\OneDrive - MOYO Business Advisory\\Documents\\Assigments\\2025-07-04 - Mine data analysis\\Data\\Combine data\\KPI FILE 19 Oct 24\\2023\\Densities Reagents 08 March 23.xlsx'


NameError: name 'file_paths' is not defined

In [103]:

all_files_df = pandas.DataFrame(buffer_list)
all_files_searched = all_files_df['File'];

all_files_df['Date'] = pandas.to_datetime(all_files_df['Date']).dt.date;
data_combined = pandas.concat([data, all_files_df]);
data_combined.sort_values(by='Date', ascending=True, inplace=True);
data_combined.drop_duplicates(by='Date', keep='First', inplace=True)
data_panel_beat = data_combined.set_index('Date', inplace=True);
data_panel_beat.to_csv("combined-data-sorted.csv");


df_buffer_dates = pandas.DataFrame(buffer_dict_empty);
df_buffer_dates['Date'] = pandas.date_range(datetime.date(2023,1,1), datetime.date.today());
df_buffer_dates['Date'] = [ date_i.date() for date_i in df_buffer_dates['Date'] ];
df_buffer_dates.sort_values(by='Date', ascending=True, inplace=True);
df_buffer_dates.set_index('Date', inplace=True);
df_buffer_dates['File'] = df_buffer_dates['File'].astype(str);


# set up logical array for data sorting 
log_arr = (df_buffer_dates.index == data_panel_beat.index[0]);

# estimate what date values are populated and which are not.
for date_i in data_panel_beat.index:
    log_arr = log_arr + (date_i == df_buffer_dates.index);

df_to_include = df_buffer_dates[~log_arr];
df_to_include['File'] = 'Interp';


DF_before_interp = pandas.concat([data_panel_beat, df_to_include])
DF_before_interp.sort_index(inplace=True);

DF_interp = DF_before_interp.copy();
DF_interp.infer_objects(copy=False); # required
DF_interp.interpolate(method='linear', limit=600, limit_direction='both', inplace=True);
DF_interp.to_csv("Interpolated_values")

t1 = time.time();
print(f"Time taken for {numpy.size(files_to_search)} files - {numpy.round((t1-t0)/60,2)} min")


TypeError: '<' not supported between instances of 'datetime.date' and 'str'

In [ ]:
files_searched = data_Frame['File'];

In [51]:
data_frame = pandas.DataFrame(buffer_list);
data_frame.to_csv("test.csv")